In [4]:
import torch, time
import torch.nn as nn
import torch.optim as optim

from src.utils import *
from src.override_resnet import *

class Args: 
    """Main function for training ResNet model.

    Args:
        --num_layers (int): number of layers
        --dataset (str): name of the dataset
        --lr (float): learning rate
        --momentum (float): momentum
        --batch_size (int): batch size
        --num_epochs (int): number of epochs
        --save_every (int): save model every n epochs
        --qat (bool): Enable Quantization Aware Training
        --only_eval (bool): only evaluation mode
        --verbose (bool): print mini-batch loss and accuracy


    Returns:
        None.
    """
    num_layers = 50
    dataset = "ImageNet"
    lr = 0.001
    momentum = 0.9
    batch_size = 128
    num_epochs = 10
    save_every = 1
    qat = True
    only_eval = True
    verbose = True
    qat_method = "static"


args = Args()

In [5]:
# %% Load the ResNet-50 model
if args.qat == True:
    if args.qat_method == "dynamic":
        # case 1 : Dynamic Quantization
        device = "cpu"
        model = resnet50_quan(
            weights=pretrained_weights_mapping[args.num_layers]
        ).to(device)
        quantized_model = torch.quantization.quantize_dynamic(
            model, {torch.nn.Linear}, dtype=torch.qint8
        )
        model = quantized_model
        print("----------Dynamic Quantization enabled")
    elif args.qat_method == "static":
        device = "cpu"
        model = resnet50_quan(weights=pretrained_weights_mapping[args.num_layers]).to(
            device
        )
        # case 2 : Static Quantization
        print("----------Static Quantization enabled")

    elif args.qat_method == "qat":
        # case 3 : Quantization Aware Training
        print("----------Quantization Aware Training enabled")
    else:
        raise ValueError("Invalid QAT method")

else:
    device = str(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
    model = layers_mapping[args.num_layers](
        weights=pretrained_weights_mapping[args.num_layers]
    ).to(device)

if args.qat == True:
    _folder_path = f"resnet{args.num_layers}_{args.dataset}_QAT"
else:
    _folder_path = f"resnet{args.num_layers}_{args.dataset}"

_file_name = f"resnet{args.num_layers}_{args.dataset}_epoch"  # resnet18_cifar10_epoch{epoch}.pth

_latest_epoch = CheckPointLoader(
    model=model,
    device=device,
    folder_path=_folder_path,
    file_name=_file_name,
    only_eval=args.only_eval,
)

# %%Set up training and evaluation processes
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
train_loader, test_loader = GetDataset(
    dataset_name=args.dataset,
    device=device,
    root="data",
    batch_size=args.batch_size,
)

----------Static Quantization enabled
No savepoint found. Starting from Pretrained Model
evaluate acc of pretrained model.


In [2]:
model.eval()
model.fuse_model()
model.qconfig = torch.quantization.get_default_qconfig("x86_64")
torch.quantization.prepare(model, inplace=True)

# 첫 번째 보정
print("Post Training Quantization Prepare: Inserting Observers")
print(
    "\n Inverted Residual Block:After observer insertion \n\n", model.features[1].conv
)

NameError: name 'model' is not defined

In [ ]:
# %% Training and evaluation
# 1cycle learning rate schedule
print("Post Training Quantization: Calibration done")

In [ ]:
torch.quantization.convert(model, inplace=True)
print("Post Training Quantization: Convert done")
print(
    "\n Inverted Residual Block: After fusion and quantization, note fused modules: \n\n",
    model.features[1].conv,
)

In [ ]:
# %% Training and evaluation
# 1cycle learning rate schedule

In [ ]:
print("Size of model after quantization")
print_size_of_model(model)